# __Использованные материалы__

* [LightFM Github](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/quickstart.html)
* [Google recommendation systems course](https://developers.google.com/machine-learning/recommendation)
* [Recommender system using Bayesian personalized ranking](https://towardsdatascience.com/recommender-system-using-bayesian-personalized-ranking-d30e98bba0b9)
* [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/)
* [How to build a Movie Recommender System in Python using LightFm](https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b)
* [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=ratings_small.csv)

# __Краткое введение__

__LightFM__ - это реализация на Python'е ряда популярных алгоритмов рекомендаций, включая эффективную реализацию BPR и WARP. Он прост в использовании, быстр (благодаря многопоточности) и дает высококачественные результаты. <br>
Существуют две основные стратегии создания рекомендательных систем: 
* __Content-based Filtering__
* __Collaborative filtering__

На практике чаще всего они используются в совокупности.<br>
<em>Далее для удобства будет использоваться термин item, который подразумевает под собой сущности, рекомендуемые системой.</em>

### __Content-based Filtering__ 
Данный подход предполагает работу с метаданными пользователя, которые собираются различными способами:
* __explicit__ - пользователь заполняет анкеты для выявление предпочтений, к примеру оценивает какой-то item по дифференцированной шкале.<br>
* __implicit__ - все действия пользователя протоколируются для выявления предпочтений, к примеру переход по ссылками, информация о компьютере пользователя и тп.<br>

### __Collaborative filtering__ 
Данный подход использует группировку пользователей и item'ов по каким-то сходствам/критериям. Будет реализоваться следующая логика "Пользователям, которым понравился item $X$, также нравились item'ы $Y$". Похожесть как правило определяется следующими методами:<br>
* __Content-based__ - на основании характеристик item'ов и пользователей.<br>
* __Transaction-based__ - на основании того, входили ли item'ы в одну транзакцию, а пользователи совершали схожие действия.<br>

### Machine-learned ranking 
В __LightFM__ представлены два классических подхода MLR'а:
* __Bayesian Personalized Ranking (BLR)__ 
* __Weighted Approximate-Rank Pairwise (WARP)__ 

### Bayesian Personalized Ranking 
Основная идея заключается в выборке и попарном сравнение положительных и отрицательных item'ов. Алгоритм в упрощенном виде можно представить следующим образом:
1. Случайным образом возьмем пользователя $u$ и item $i$, который ранее был выбран пользователем, в таком случае item $i$ будет считаться <em>положительным.</em>
2. Случайным образом возьмем item $j$, который был выбран пользователем <em>реже</em>, чем $i$ (в том числе, который пользователь никогда не выбирал), в таком случае item $j$ будет считаться <em>отрицательным.</em>
3. Вычисляем оценку $p_{ui}$ и $p_{uj}$ пользователя $u$, а также положительного item'а $i$ и отрицательного item'а $j$ соответственно.
4. Находим разницу между положительными и отрицательными оценками, как $x_{uij} = p_{ui} - p_{uj}.$ 
5. Пропускаем эту разницу через сигмоид и используем ее для вычисления веса для обновления всех параметров модели с помощью градиентного шага(SGD).

### Weighted Approximate-Rank Pairwise
Концепция данного подхода схожа с BPR, за исключением случаев, когда происходит градиентный шаг:
* В BPR градиентный шаг происходит каждый раз с разницей в качестве веса.
* WARP совершает градиентный шаг только в случае неверного предсказания (т.е. оценка отрицательного item'а больше положительного). Если предсказание было верным, то продолжаем выбирать отрицательные item'ы, пока не получим неверный прогноз или не достигнем некоторого порогового значения.

Для этих целей WARP предоставляет два гиперпараметра:
1. __Margin__ - определяет насколько ошибочным должен быть прогноз для совершения градиентного шага. 
2. __Cutoff__ - определяет сколько раз мы готовы выбирать отрицательные примеры, пытаясь получить неверное предсказание, прежде чем откажемся и перейдем к следующему пользователю.

<em>Автор статьи [Learning to Rank Sketchfab Models with LightFM](https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/) утверждает, что на практике вероятнее всего WARP предпочтительнее для большинства рекомендательных систем, нежели BPR.</em>

# __Тестовый пример__
Попробуем реализовать простейшую рекомендательную систему на основе [датасета, предоставляемого LightFM'ом.](https://grouplens.org/datasets/movielens/100k/)
## __Установка зависимостей__
### __Виртуальное окружение__
Для его создания будет использоваться conda.
#### Установка conda для Windows:

In [ ]:
%%cmd
@"%SystemRoot%\System32\WindowsPowerShell\v1.0\powershell.exe" -NoProfile -InputFormat None -ExecutionPolicy Bypass -Command "[System.Net.ServicePointManager]::SecurityProtocol = 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))" && SET "PATH=%PATH%;%ALLUSERSPROFILE%\chocolatey\bin"
ECHO Y | choco install miniconda3 --params="'/AddToPath:1'"

#### Установка conda для Ubuntu:

In [ ]:
%%sh
sudo apt update --yes
sudo apt upgrade --yes

wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda.sh
bash ~/miniconda.sh -b -p ~/miniconda 
rm ~/miniconda.sh

export PATH=~/miniconda/bin:$PATH

#### Создаем и активируем виртуальное окружение c помощью команд <br>
```
conda create -n LightFM-env
conda activate LightFM-env
pip install --user ipykernel
python -m ipykernel install --user --name=LightFM-env
```
#### Затем добавляем новый кернел в нотбук
<em>По неведомым мне причинам подход к созданию виртуальной среды через conda в Windows упорно не хотел работать и активация виртуального окружения происходила только через cmd (Powershell отказывался работать). Полдня стараний зафиксить эту проблему не увенчались успехом, соответственно терминал был не доступен, поэтому дальше будет описан подход через venv.

In [6]:
import sys
from platform import python_version
if float(python_version()[:-2]) < 3.3: #Поскольку venv является стандартной библиотекой в Python начиная с версии 3.3.*
    print("Upgrade Python to use venv library features for correct further work")

In [14]:
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m venv LightFM-env

#### Windows

In [18]:
%%cmd
.\LightFM-env\Scripts\activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --name=LightFM-env
jupyter-notebook

Microsoft Windows [Version 10.0.19042.1165]
(c) Љ®аЇ®а жЁп Њ ©Єа®б®дв (Microsoft Corporation). ‚бҐ Їа ў  § йЁйҐ­л.

C:\A.Mindset\internship_ds\LightFM>.\LightFM-env\Scripts\activate

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>python -m pip install --upgrade pip
  Using cached pip-21.2.4-py3-none-any.whl (1.6 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>pip install ipykernel

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>python -m ipykernel install --name=LightFM-env
Installed kernelspec LightFM-env in C:\ProgramData\jupyter\kernels\lightfm-env

(LightFM-env) C:\A.Mindset\internship_ds\LightFM>

In [4]:
!jupyter kernelspec list

Available kernels:
  lightfm-env    C:\Users\fire9\AppData\Roaming\jupyter\kernels\lightfm-env
  python3        c:\a.mindset\internship_ds\lightfm\lightfm-env\share\jupyter\kernels\python3


#### Ubuntu

In [ ]:
%%sh
source LightFM-env/bin/activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --name=LightFM-env
jupyter-notebook

In [ ]:
!jupyter kernelspec list

#### Затем добавляем новый кернел в нотбук
### Установка библиотек для тестового примера

In [5]:
!pip install lightfm
!pip install numpy
!pip install scipy

  Using cached lightfm-1.16.tar.gz (310 kB)
  Using cached scipy-1.7.1-cp37-cp37m-win_amd64.whl (33.6 MB)
  Using cached scikit_learn-0.24.2-cp37-cp37m-win_amd64.whl (6.8 MB)
  Using cached threadpoolctl-2.2.0-py3-none-any.whl (12 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
Using legacy 'setup.py install' for lightfm, since package 'wheel' is not installed.
    Running setup.py install for lightfm: started
    Running setup.py install for lightfm: finished with status 'done'


In [3]:
#Импорт необходимых библиотек для тестового примера
import numpy as np
from lightfm.datasets import fetch_movielens #метод lightfm для извлечения данных фильма
from lightfm import LightFM

In [55]:
#Получаем данные фильма с минимальным рейтингом 4
data = fetch_movielens(min_rating = 4.0)

#Отобразим обучающий и тестовый набор
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [30]:
#Создадим модель
model = LightFM(loss = 'warp')
#Тренировка
model.fit(data['train'], epochs=30, num_threads=2)

#Рекомендательная функция
def sample_recommendation(model, data, user_ids):
    #Число пользователей и фильмов в обучающем наборе
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
    	#Фильмы, которые уже понравились пользователям
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        #Предсказание фильмов, которые им понравится
        scores = model.predict(user_id, np.arange(n_items))
        #Сортирует результат по оценке
        top_items = data['item_labels'][np.argsort(-scores)]
        #Отображение результатов
        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 451])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        L.A. Confidential (1997)
        Starship Troopers (1997)
        Cop Land (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        Fargo (1996)
        English Patient, The (1996)
        Contact (1997)
User 451
     Known positives:
        Twelve Monkeys (1995)
        Babe (1995)
        Mr. Holland's Opus (1995)
     Recommended:
        Raiders of the Lost Ark (1981)
        Amadeus (1984)
        Sting, The (1973)


# __Работа с kaggle датасетом__
Для этих целей возьмем датасет [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset/home?select=keywords.csv)

In [11]:
!pip install kaggle

Теперь нам необходимо создать API токен на kaggle по адресу `https://www.kaggle.com/<username>/account` и поместить его в папку .kaggle, расположение которой зависит от ОС:
* Для Windows - `C:\Users\<Windows-username>\.kaggle\kaggle.json`
* Для Linux систем - ```~/.kaggle/kaggle.json```

### Скачивание и распаковка датасета

In [33]:
from zipfile import ZipFile
import os
!mkdir LightFM-Dataset 
%cd .\LightFM-Dataset
!kaggle datasets download -d rounakbanik/the-movies-dataset
zip_file = ZipFile('the-movies-dataset.zip')
zip_file.extractall()
zip_file.close()
os.remove("the-movies-dataset.zip")
%cd ..

C:\A.Mindset\internship_ds\LightFM\LightFM-Dataset



  0%|          | 0.00/228M [00:00<?, ?B/s]
  0%|          | 1.00M/228M [00:00<00:29, 8.14MB/s]
  2%|2         | 5.00M/228M [00:00<00:10, 22.4MB/s]
  4%|3         | 8.00M/228M [00:00<00:09, 24.9MB/s]
  5%|4         | 11.0M/228M [00:00<00:09, 25.0MB/s]
  8%|8         | 19.0M/228M [00:00<00:05, 42.1MB/s]
 12%|#2        | 28.0M/228M [00:00<00:03, 57.6MB/s]
 15%|#5        | 35.0M/228M [00:00<00:03, 60.3MB/s]
 18%|#7        | 41.0M/228M [00:01<00:04, 41.6MB/s]
 20%|##        | 46.0M/228M [00:01<00:05, 34.1MB/s]
 22%|##1       | 50.0M/228M [00:01<00:06, 30.5MB/s]
 24%|##3       | 54.0M/228M [00:01<00:06, 28.1MB/s]
 25%|##5       | 58.0M/228M [00:01<00:06, 26.4MB/s]
 27%|##6       | 61.0M/228M [00:01<00:06, 26.3MB/s]
 28%|##8       | 64.0M/228M [00:02<00:06, 27.2MB/s]
 30%|##9       | 68.0M/228M [00:02<00:05, 28.2MB/s]
 31%|###1      | 71.0M/228M [00:02<00:06, 27.0MB/s]
 32%|###2      | 74.0M/228M [00:02<00:05, 27.0MB/s]
 34%|###3      | 77.0M/228M [00:02<00:06, 26.2MB/s]
 35%|###5      | 80.


C:\A.Mindset\internship_ds\LightFM


In [35]:
!jupyter kernelspec list

Available kernels:
  lightfm-env    C:\Users\fire9\AppData\Roaming\jupyter\kernels\lightfm-env
  python3        c:\a.mindset\internship_ds\lightfm\lightfm-env\share\jupyter\kernels\python3
